In [ ]:
import time
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model("exported_model/model.h5")

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)
pTime = 0

In [ ]:
while True:
    success, imgRGB = cap.read()
    resized = cv2.resize(imgRGB, (192, 108))  # Resize according to your model input shape

    input_tensor = resized / 255.0  # Normalize if required

    input_tensor = np.expand_dims(input_tensor, axis=0)

    yhat = model.predict(input_tensor)

    s_coords = yhat[0]
    s_img = np.copy(imgRGB)
    keys_coords = list(np.multiply(list(zip(s_coords[::2], s_coords[1::2])), [640, 360]))

    for i, (x, y) in enumerate(keys_coords):
        cv2.circle(s_img, (int(x), int(y)), 8, (255, 0, 0), -1)
        if i < len(keys_coords) - 1:
            cv2.line(s_img, (int(x), int(y)), (int(keys_coords[i + 1][0]), int(keys_coords[i + 1][1])), (0, 255, 0), 3)

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    s_img = cv2.flip(s_img, 1)
    cv2.putText(s_img, str(int(fps)), (70, 70), cv2.FONT_HERSHEY_DUPLEX, 3, (255, 0, 0), 3)

    cv2.imshow("Video", s_img)
    cv2.waitKey(1)